In [4]:
import cv2
import numpy as np

net = cv2.dnn.readNet("data/yolov2.weights", "data/yolov2.cfg")

with open("data/coco.names", "r") as f:
    classes = f.read().strip().split("\n")

cascade = cv2.CascadeClassifier('data/HaarCascadeClassifier/car4.xml')

video_capture = cv2.VideoCapture("videos/traffic-video.mp4")

if not video_capture.isOpened():
    print("Error: Cannot access the video source.")
    exit()

bbox = None
tracking = False

while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    if not tracking:
        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        layer_names = net.getUnconnectedOutLayersNames()
        detections = net.forward(layer_names)

        for detection in detections:
            for obj in detection:
                scores = obj[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.5:
                    center_x, center_y, width, height = (obj[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]))
                    x, y = int(center_x - width/2), int(center_y - height/2)

                    label = f"{classes[class_id]}: {confidence:.2f}"
                    top_left = (int(x), int(y))
                    bottom_right = (int(x + width), int(y + height))
                    cv2.rectangle(frame, tuple(top_left), tuple(bottom_right), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    else:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        obj = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=35, minSize=(20, 20))

        for (x, y, w, h) in obj:
            top_left = (int(x), int(y))
            bottom_right = (int(x + width), int(y + height))
            cv2.rectangle(frame, tuple(top_left), tuple(bottom_right), (0, 255, 0), 2)
            cv2.putText(frame, "Tracking", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Object Detected",frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()

cv2.destroyAllWindows()
